![PHENIX](logoPHENIX.png)

## 5) Python as a wrapper for other software.

In this part we'll focus on the uses of python as a tool to carry out simulations in materials science, chemistry and physics.

**N.B.** To follow this tutorial you should install [pycp2k](https://github.com/SINGROUP/pycp2k).

**N.B.** This is a modified version of the examples provided by the creators of pycp2k.

## pycp2k - A python interface for cp2k

Create the Si lattice here as an [ASE](https://wiki.fysik.dtu.dk/ase/) Atoms object. We will use it later on to
automatically create entries in the CP2K input. Here we use ASE specific
function to do the job. One may also load any of the ase supported structure
files by using `ase.io.read()` or use the ASE Atoms and Atom classes directly
to create the structure.

In [1]:
import re
from pycp2k import CP2K
from ase.lattice.cubic import Diamond

lattice = Diamond(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                  symbol='Si',
                  latticeconstant=5.430697500,
                  size=(1, 1, 1))

Setup directories and mpi parallelization. If you specify a working directory
and a project name, the output and input file names and the
GLOBAL.Project_name keyword are automatically generated for you. You can
alternatively specify each separately. You can setup the cp2k command with
calc.cp2k_command, and the mpi command with calc.mpi_command.  MPI can be
turned on or off with calc.mpi_on (on by default). The -n flag can be setup
with calc.mpi_n_processes. Any special flags can be specified by using
calc.cp2k_flags or calc.mpi_flags. By default cp2k is run in the output
directory, but you can change the working directory with

In [2]:
# calc.working_directory.
calc = CP2K()
calc.working_directory = "./"
calc.project_name = "si_bulk"
calc.mpi_n_processes = 1

In [3]:
#===============================================================================
# Create shortcuts for the most used subtrees of the input. You don't have to
# specify these but they will make your life easier.
CP2K_INPUT = calc.CP2K_INPUT  # This is the root of the input tree
GLOBAL = CP2K_INPUT.GLOBAL
# Repeatable sections are added with X_add() function. Optionally you can
# provide the Section_parameter as an argument to this function.
FORCE_EVAL = CP2K_INPUT.FORCE_EVAL_add()
SUBSYS = FORCE_EVAL.SUBSYS
DFT = FORCE_EVAL.DFT
SCF = DFT.SCF

Fill input tree. Section names are in upper case, keywords are capitalized.
Most IDEs will be able to automatically suggest the entries as you begin
typing them. Relevant parts of the documentation have been copied to
parsedclasses.py and some IDEs provide quick access to them (e.g. in spyder
you can search documentation for keyword with the command "go to definition"

The keywords can entered as any python type that converts to string (int,
float, etc.). Additionally you can provide non-repeatable keywords as lists.
In that case the each list element is converted to string, separated with
white space and printed into input file. Repeatable keywords can also be
defined as lists, but in this case each list item corresponds to a new
repeated item. For an example of these features see examples/example_qmmm.py.

In [4]:
GLOBAL.Run_type = "ENERGY_FORCE"
GLOBAL.Print_level = "MEDIUM"

These utility functions will create entries to the input tree from the ASE
Atoms object created earlier. As arguments these two functions take the
subsys where the entries should be created and the Atoms object from which
they are exctracted.

In [ ]:
calc.create_cell(SUBSYS, lattice)
calc.create_coord(SUBSYS, lattice)

FORCE_EVAL.Method = "Quickstep"
FORCE_EVAL.PRINT.FORCES.Section_parameters = "ON"
FORCE_EVAL.PRINT.FORCES.Filename = "forces"
DFT.Basis_set_file_name = "BASIS_SET"
DFT.Potential_file_name = "GTH_POTENTIALS"
DFT.QS.Eps_default = 1.0E-10
DFT.MGRID.Ngrids = 4
DFT.MGRID.Cutoff = 300
DFT.MGRID.Rel_cutoff = 60
DFT.XC.XC_FUNCTIONAL.Section_parameters = "PADE"
SCF.Scf_guess = "ATOMIC"
SCF.Eps_scf = 1.0E-7
SCF.Max_scf = 300
SCF.DIAGONALIZATION.Section_parameters = "ON"
SCF.DIAGONALIZATION.Algorithm = "STANDARD"
SCF.MIXING.Section_parameters = True
SCF.MIXING.Method = "BROYDEN_MIXING"
SCF.MIXING.Alpha = 0.4
SCF.MIXING.Nbroyden = 8
FORCE_EVAL.PRINT.FORCES.Section_parameters = "ON"

KIND = SUBSYS.KIND_add("Si")  # Section_parameters can be provided as argument.
KIND.Basis_set = "DZVP-GTH-PADE"
KIND.Potential = "GTH-PADE-q4"

In [ ]:
#===============================================================================
# After you have created your simulation you can choose how to run it.
# Typically there are two options:

# 1. Only write the input file. CP2K is then run manually or with some other
# script.
calc.write_input_file()

# 2. Write the input file and run CP2K as a subprocess in python.
calc.run()

#===============================================================================
# You can analyse the output with whatever tool you want. E.g. the final energy
# can simply be found with a regular expression:
with open(calc.output_path, "r") as fin:
    regex = re.compile(" ENERGY\| Total FORCE_EVAL \( QS \) energy \(a\.u\.\):\s+(.+)\n")
    for line in fin:
        match = regex.match(line)
        if match:
            print("Final energy: {}".format(match.groups()[0]))

|============================ PYCP2K RUN STARTED ==============================|
  >> CP2K version check...

        |========================= WARNING ===========================|
        |  The CP2K version does not match the version for which       |
        |  PYCP2K was configured. This will affect the availability    |
        |  of keywords and sections in the input tree!                 |
        |--------------------------------------------------------------|

  >> Creating CP2K input file...
  >> Performing syntax check on input file...
  >> Running CP2K:
     -CP2K version: b'3.0'
     -CP2K revision: b'svn:16458'
     -CP2K command: cp2k-ubuntu.popt
     -MPI command: mpirun
     -Processes: 1


We can also set up a script to perform multiple simulations. In the example below we search for the best energy cut-off.

In [ ]:
#===============================================================================
# Search for a good CUTOFF
GLOBAL.Run_type = "ENERGY"
GLOBAL.Print_level = "LOW"
energies = []
for cutoff in range(40, 90, 20):
    DFT.MGRID.Cutoff = cutoff
    calc.output_path = calc.working_directory + "/" + calc.project_name + str(cutoff) + ".out"
    calc.run()
    with open(calc.output_path, "r") as fin:
        regex = re.compile(" ENERGY\| Total FORCE_EVAL \( QS \) energy \(a\.u\.\):\s+(.+)\n")
        for line in fin:
            match = regex.match(line)
            if match:
                energies.append(match.groups()[0])

print(energies)